In [8]:
%matplotlib inline
import pandas as pd
import numpy as np
import xarray as xr
import glob
from netCDF4 import Dataset as nc, num2date, MFDataset as mc, date2num
import os
import pytz
from matplotlib import pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import seaborn as sns
import cufflinks as cf
cf.set_config_file(offline=False, world_readable=True, theme='ggplot')
sns.set_style("whitegrid")
from DataFrame import *
from datetime import datetime
from astropy.stats import LombScargle
import astropy.units as u
from scipy import signal
import matplotlib, sys
from mpl_toolkits.basemap import Basemap, cm as colmap
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [30.0,30.0]
from collections import namedtuple
timezone = pytz.timezone('Australia/Darwin')
utc = pytz.UTC
from matplotlib import animation, rc
from matplotlib.colors import LightSource
from matplotlib.colorbar import cm as colmap2
from matplotlib.patches import Polygon

In [2]:
pd.__file__

'/usr/local/lib/python3.5/dist-packages/pandas/__init__.py'

#### Define some paths

In [3]:
CPOLF = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL_TIWI_1998-2017-old.nc')
WOHF = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL_WOH.nc')
CMORPHF = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'UM', 'CMORPH_V1_WOH.nc')
WOHFv1 = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL_WOHv1.nc')
WOHFv2 = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL_WOHv2.nc')
BurstF = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL_burst-cmorph.pkl')
PercF = os.path.join(os.environ['HOME'],'Data','Extremes','CPOL','CPOL_TIWI_Percentiles-old.hdf5')
extremeTS = os.path.join(os.environ['HOME'],'Data','Extremes','CPOL','CPOL_TIWI_TS_extrme.pkl')
percf =  os.path.join(os.environ['HOME'],'Data','Extremes','CPOL', 'Periods.npz')
outdir = os.path.join(os.getenv('HOME'),'Data','Extremes','UM', 'Plot', 'Week_of_Hector')
geodataf = os.path.join(os.getenv('HOME'), 'Data', 'Extremes', 'UM')
trackfile = 'tint_tracks_2006_11_10_00-2006_11_18_23.pkl'
CPOL_trackfile = os.path.join(os.getenv('HOME'),'Data', 'Extremes','CPOL', 'Tracking', trackfile)
remap_res = '2.5km'
UMdir = os.path.join(os.getenv('HOME'), 'Data', 'Extremes', 'UM', 'darwin', 'RA1T')

In [227]:
def get_mean(data_file, res=2.5):
    if isinstance(data_file, str):
        dataset = pd.read_pickle(data_file)
    else:
        dataset = data_file
    dataset = dataset[dataset.isolated][['max', 'mean', 'area','time']]
    uid = np.array(dataset.index.levels[-1]).astype('i')
    uid.sort()
    columns=['avg_mean','avg_area', 'avg_max', 'max_mean', 'max_area', 'max_max',
             'std_mean', 'std_area', 'std_max','dur']
    values=np.empty((1,len(columns)))
    index = []
    for nn, idx in enumerate(uid):
        df = pd.DataFrame(dataset.xs(str(idx), level='uid').values, columns=dataset.columns)
        if len(df):
            index.append(idx)
            mean = df[['mean','area','max']].mean().values
            Max = df[['mean','area','max']].max().values
            std = df[['mean', 'area', 'max']].std().values
            dur = np.array([(df['time'].max() - df['time'].min()).total_seconds()/60.]).round(0)
            values= np.append(values, np.r_[mean, Max, std, dur][np.newaxis],axis=0)
    out = pd.DataFrame(values[1:,], index=index, columns=columns)
    for area in ('avg_area', 'max_area'):
        out[area] = out[area] * res**2
    return out

### Define the ensembles

In [4]:
ensembles = ('20061109T1200Z', '20061109T1800Z', '20061110T0000Z',
             '20061110T0600Z', '20061110T1200Z', '20061110T1800Z',
             '20061111T0000Z', '20061111T1200Z')


In [228]:
Simulation = namedtuple('Simulation', 'dataset resolution')
UM133 = Simulation(dataset={}, resolution=1.33)
UM044 = Simulation(dataset={}, resolution=0.44)
OBS = Simulation(dataset=get_mean(CPOL_trackfile), resolution=2.55)
for ens in ensembles:
        for UM in (UM133, UM044):
            track_f = glob.glob(os.path.join(UMdir, ens, 'darwin', '1p33km', 'Tracking', '*.pkl'))[0]
            UM.dataset[ens] =get_mean(pd.read_pickle(track_f))
            #UM.dataset.append(pd.read_pickle(track_f))

In [230]:
UM044.dataset

{'20061109T1200Z':     avg_mean    avg_area    avg_max   max_mean  max_area    max_max  std_mean  \
 0   6.133280  245.703125  19.463623   8.779212    375.00  33.031250  2.286252   
 1   2.073547   46.875000   3.794271   3.145833     68.75   6.769531  0.553876   
 2   2.957129   56.250000   5.984375   4.147656     62.50   8.683594  1.683660   
 3   2.388616   68.750000   4.162109   2.904297     93.75   5.535156  0.616845   
 4   5.204001  243.750000  15.600260   9.602163    431.25  24.644531  2.354647   
 5   2.671840   59.375000   6.844727   2.970052     81.25   9.445312  0.302509   
 6   4.156795   77.272727   9.719105   5.793870    100.00  16.132812  1.345470   
 8   3.138166   86.250000   6.891406   4.629102    125.00  13.304688  1.083256   
 9   4.721315   65.000000   9.588281   6.246484     81.25  13.347656  1.816520   
 10  5.040971   50.892857  11.426897   7.054332     68.75  18.722656  1.568613   
 11  5.177229   76.785714  14.263393   7.299479    118.75  22.199219  1.750897  

In [201]:
UM044.dataset

[    avg_mean  avg_area    avg_max   max_mean  max_area    max_max    dur
 0   6.133280       0.0  19.463623   8.779212       0.0  33.031250  150.0
 1   2.073547       0.0   3.794271   3.145833       0.0   6.769531   50.0
 2   2.957129       0.0   5.984375   4.147656       0.0   8.683594   10.0
 3   2.388616       0.0   4.162109   2.904297       0.0   5.535156   30.0
 4   5.204001       0.0  15.600260   9.602163       0.0  24.644531  110.0
 5   2.671840       0.0   6.844727   2.970052       0.0   9.445312   30.0
 6   4.156795       0.0   9.719105   5.793870       0.0  16.132812  100.0
 8   3.138166       0.0   6.891406   4.629102       0.0  13.304688   40.0
 9   4.721315       0.0   9.588281   6.246484       0.0  13.347656   40.0
 10  5.040971       0.0  11.426897   7.054332       0.0  18.722656   60.0
 11  5.177229       0.0  14.263393   7.299479       0.0  22.199219   60.0
 12  3.040946       0.0   6.095703   3.783203       0.0   8.828125   30.0
 13  4.939261       0.0   9.505208   6

In [197]:
df = get_mean(UM044.dataset[0])
df

,avg_mean,avg_area,avg_max,max_mean,max_area,max_max,dur
0,6.133280,0.0,19.463623,8.779212,0.0,33.031250,150.0
1,2.073547,0.0,3.794271,3.145833,0.0,6.769531,50.0
2,2.957129,0.0,5.984375,4.147656,0.0,8.683594,10.0
3,2.388616,0.0,4.162109,2.904297,0.0,5.535156,30.0
4,5.204001,0.0,15.600260,9.602163,0.0,24.644531,110.0
5,2.671840,0.0,6.844727,2.970052,0.0,9.445312,30.0
6,4.156795,0.0,9.719105,5.793870,0.0,16.132812,100.0
8,3.138166,0.0,6.891406,4.629102,0.0,13.304688,40.0
9,4.721315,0.0,9.588281,6.246484,0.0,13.347656,40.0
10,5.040971,0.0,11.426897,7.054332,0.0,18.722656,60.0
